In [1]:
import numpy as np
from mpe2 import simple_spread_v3
from pettingzoo.utils.conversions import aec_to_parallel

from failure_api.wrappers import CommunicationWrapper
from failure_api.communication_models import (
    ProbabilisticModel,
    BaseMarkovModel,
    DistanceModel,
)

def test_masking_variability(model_class, model_args={}, model_name="UnknownModel", N=3, runs=5):
    print(f"\n🔎 Testing variability of: {model_name} ({model_class.__name__})")
    matrices = []

    for i in range(runs):
        # Fresh environment for each test run
        env = simple_spread_v3.env(N=N)
        env.reset(seed=42 + i)

        # Special case for DistanceModel: define pos_fn dynamically
        if model_class is DistanceModel:
            def get_positions(env_ref):
                return {
                    agent: env_ref.env.state()[idx][:2]
                    for idx, agent in enumerate(env_ref.possible_agents)
                }
            pos_fn = lambda env_ref=env: get_positions(env_ref)
            model_args["pos_fn"] = pos_fn

        model = model_class(agent_ids=env.possible_agents, **model_args)
        wrapped = CommunicationWrapper(env, failure_models=[model])
        wrapped.reset(seed=42 + i)

        matrix = np.round(wrapped.comms_matrix.matrix.toarray(), 2) 
        matrices.append(matrix)
        print(f"Run {i+1}:\n{matrix}")

    # Check if at least one matrix differs from the others
    unique = any(not np.array_equal(matrices[0], m) for m in matrices[1:])
    if unique:
        print(f"✅ {model_name} shows variation across runs.\n")
    else:
        print(f"❌ {model_name} is deterministic! No variation across runs.\n")

# Run tests for all three models
test_masking_variability(
    ProbabilisticModel,
    model_args={"failure_prob": 0.5},
    model_name="ProbabilisticModel"
)

agent_ids = [f"agent_{i}" for i in range(3)]
tp_matrix = np.array([[0.8, 0.2], [0.2, 0.8]])

transition_probs = {
    (a1, a2): tp_matrix.copy()
    for a1 in agent_ids
    for a2 in agent_ids
    if a1 != a2
}

test_masking_variability(
    BaseMarkovModel,
    model_args={
        "transition_probabilities": transition_probs,
        "default_matrix": np.array([[0.8, 0.2], [0.2, 0.8]])  # ✅ Explicit default_matrix
    },
    model_name="BaseMarkovModel"
)

test_masking_variability(
    DistanceModel,
    model_args={"distance_threshold": 1.0},  # pos_fn injected inside
    model_name="DistanceModel"
)



🔎 Testing variability of: ProbabilisticModel (ProbabilisticModel)


AttributeError: 'numpy.ndarray' object has no attribute 'toarray'